# Atelier 1

Cet atelier propose une initiation à la bioinformatique appliquée à l’étude du microbiome des sols. Les participants découvriront les concepts clés, les formats de données et les outils couramment utilisés. L’analyse de la diversité fongique et eucaryote sera abordée à travers un pipeline complet, incluant les bases de données de référence et la visualisation des résultats. Une introduction aux statistiques appliquées aux données omiques complètera la formation, avec un focus sur les indices de diversité (alpha/bêta) et les analyses multivariées de base.

In [ ]:
!git clone https://github.com/thomasbio/AtelierSYMBIOSOL.git

In [ ]:
%cd AtelierSYMBIOSOL
%run setup_qiime2


## Importer les informations relatives aux échantillons (metadata)

In [ ]:
import pandas as pd
metadata = pd.read_csv('data/metadata.tsv', sep = '\t')
metadata

## Importer les fichiers FASTQ (R1 et R2)

In [ ]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path data/procaryotes/fastq_subset \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

## Évaluer la qualité des séquences et définir les paramètres de filtration

In [ ]:
!qiime demux summarize \
  --i-data demux-paired-end.qza \
  --o-visualization demux-paired-end.qzv


In [ ]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux-paired-end.qza \
  --o-table table-emp \
  --o-representative-sequences rep-seqs \
  --o-denoising-stats denoising-stats.qza \
  --p-n-threads 1 \
  --p-trim-left-f 19 \
  --p-trim-left-r 20 \
  --p-trunc-len-f 260 \
  --p-trunc-len-r 230

# Visualisation des stats

In [ ]:
qiime metadata tabulate \
  --m-input-file denoising-stats.qza \
  --o-visualization denoising-stats.qzv

## Assignation taxonomique

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier 2024.09.backbone.v4.nb.sklearn-1.4.2.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxo-gg2.qza

## Visualiser les proportions relatives des groupes taxonomiques

In [ ]:
!qiime taxa barplot \
  --i-table table-emp.qza \
  --i-taxonomy taxo-gg2.qza \
  --m-metadata-file data/metadata.tsv \
  --o-visualization taxa-bar-plots-emp.qzv

#qiime2.Visualization.load('taxa-bar-plots-emp.qzv')

## Diversité alpha

In [ ]:
!qiime feature-table rarefy \
  --i-table table-emp.qza \
  --p-sampling-depth 4000 \
  --o-rarefied-table rarefied4k-table.qza

# [shannon, chao1, observed_features, pielou_e]
!qiime diversity alpha \
  --i-table rarefied4k-table.qza \
  --p-metric shannon \
  --o-alpha-diversity shannon_4k.qza

!qiime diversity alpha-group-significance \
  --i-alpha-diversity shannon_4k.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization shannon-group-significance.qzv

## Diversité beta

In [ ]:
!qiime diversity beta \
  --i-table table-emp.qza \
  --p-metric braycurtis \
  --o-distance-matrix bray_curtis_distance.qza

!qiime diversity pcoa \
  --i-distance-matrix bray_curtis_distance.qza \
  --o-pcoa bray_curtis_pcoa.qza

!qiime emperor plot \
  --i-pcoa bray_curtis_pcoa.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization bray_curtis_emperor.qzv

!qiime diversity beta-group-significance \
  --i-distance-matrix bray_curtis_distance.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Crop \
  --o-visualization bray_curtis_permanova.qzv \
  --p-pairwise